# BTTH05: Ẩn dữ liệu trên ảnh bằng phương pháp LSB-OPAP

TODO: Võ Phương Hòa - 1412192

---

## 1. Cách làm bài và nộp bài

**Làm bài**

Bạn sẽ làm trực tiếp trên file notebook này; trong file, mình đã để từ `TODO` để cho biết những chỗ mà bạn cần phải làm (trong đó, `TODO` đầu tiên là bạn phải ghi họ tên và MSSV vào phần đầu của file).

*Mục tiêu chính ở đây là học, học một cách chân thật; nếu bạn được 5 điểm nhưng bạn làm bài một cách chân thật thì vẫn tốt hơn nhiều so với 10 điểm mà không chân thật. Bạn có thể thảo luận với các bạn khác; nhưng bài làm phải là của chính bạn, dựa trên sự hiểu của chính bạn, lời văn là của chính bạn, code là của chính bạn. Bạn không nên đọc bài làm của các bạn năm trước. Nếu bị phát hiện gian lận thì bạn sẽ bị 0 điểm cho toàn bộ phần thực hành.*

**Nộp bài**

Khi chấm bài, đầu tiên mình sẽ chọn `Kernel` - `Restart & Run All` để restart và chạy tất cả các cell trong notebook của bạn; do đó, trước khi nộp bài, bạn nên chạy thử `Kernel` - `Restart & Run All` để đảm bảo mọi chuyện diễn ra đúng như mong đợi.

Sau đó, trong thư mục `MSSV` (vd, nếu bạn có MSSV là 1234567 thì bạn đặt tên thư mục là `1234567`) bạn đặt: file `Ex05-DataHiding-Img-LSB_OPAP.ipynb`, và các file dữ liệu mà mình đính kèm (`cover.bmp`, `msg.txt`); rồi nén thư mục `MSSV` này lại và nộp ở link trên moodle.

## 2. Import

In [1]:
%matplotlib inline
import numpy as np
import bitarray
from scipy.misc import imread, imsave
import matplotlib.pyplot as plt
# You can also import other things ...

## 3. Hàm nhúng

In [2]:
def embed_lsb(cover_img_file, msg_file, num_lsbs, stego_img_file, opap):
    """
    Embeds a message into a cover image using the LSB method.
    
    Parameters
    ----------
    cover_img_file : string
        The name of the cover img file.
    msg_file : string
        The name of the message file.
    num_lsbs : int
        The number of LSBs used to embed.
    stego_img_file : string
        The name of the stego img file.
    opap : bool
        Indicating whether OPAP (Optimal Pixel Adjustment Process) is applied to stego img or not.
        If opap is True, you need to print out:
            - MAE (Mean Absolute Error) between stego img and cover img before OPAP.
            - MAE (Mean Absolute Error) between stego img and cover img after OPAP.
            - The percentage of stego img's pixels falling into case 1 and 4
              (If stego img has 10 pixels, and 2 of them fall into case 1, then:
              the percentage of stego img's pixels falling into case-1 = 2*100/10 = 20%).
            
            Example: `MAE before OPAP: 100, MAE after OPAP: 50, case 1: 20%, case 4: 0%`.
    """
    # TODO
     # 1. Read cover img file
    cover_img = imread(cover_img_file)
        
    # 2. Read msg file and process
    # 2.1. Read msg file
    f = open(msg_file, 'r')
    msg_text = f.read()
    f.close()
    
    # 2.2 Convert msg to msg bits
    msg_bits = bitarray.bitarray()
    msg_bits.frombytes(msg_text)
    
    # 2.3. Check if it is possible to embed
    len_msg_bits = len(msg_bits)
    max_num_bits = num_lsbs * cover_img.size
    if (len_msg_bits >= max_num_bits):
        return False
    
    # 2.4. Add '100...' to msg bits
    msg_bits.extend('1' + '0' * (max_num_bits - len_msg_bits - 1))
   
    # 2.5. Convert msg bits to msg img
    msg_bits_str = msg_bits.to01()
    msg_img = np.array([int(msg_bits_str[i: i + num_lsbs], 2) for i in range(0, len(msg_bits), num_lsbs)])
    msg_img = msg_img.reshape(cover_img.shape)
    
    # 3. Embed msg img into cover img
    stego_img = (cover_img >> num_lsbs << num_lsbs) + msg_img    
    
    # 4. If OPAP = True, change the pixels in region 1 and 4
    # to reduce the difference between stego img and cover img
    if opap == True:
        err_img = np.int16(stego_img) - np.int16(cover_img)
        case1_map = (err_img > 2 ** (num_lsbs - 1)) * (stego_img >= 2 ** num_lsbs)
        case4_map = (err_img < -2 ** (num_lsbs - 1)) * (stego_img < 256 - 2  ** num_lsbs)
        
        # MAE (Mean Absolute Error) between stego img and cover img before OPAP
        mae_1 = np.mean(np.abs(stego_img - cover_img))
        
        # Change the pixels in case 1 and 4
        stego_img[case1_map] -= 2 ** num_lsbs
        stego_img[case4_map] += 2 ** num_lsbs
        
        # MAE (Mean Absolute Error) between stego img and cover img after OPAP
        mae_2 = np.mean(np.abs(stego_img - cover_img))
        
        # Find num of pixels in case 1 and 4
        sum_1 = 0
        sum_4 = 0
        leng = len(case1_map)
        for i in range(leng):
            for j in range(leng):
                if case1_map[i, j] == True:
                    sum_1 += 1
                if case4_map[i, j] == True:
                    sum_4 += 1
    
        # The percentage of stego img's pixels falling into case 1 and 4
        sum_of_pixels = stego_img.size
        per_1 = sum_1 * 100 / sum_of_pixels
        per_4 = sum_4 * 100 / sum_of_pixels
        
        # Print results
        print 'MAE before OPAP:', mae_1, ', MAE after OPAP:', mae_2, ', case 1:', per_1, '% , case 4:', per_4, '%'
                
    # 5. Write stego img to file
    imsave(stego_img_file, np.asarray(stego_img).astype('uint8')) 

    return True

## 4. Hàm rút trích (giữ nguyên hàm tuần trước)

### 4.1. Hàm chuyển đổi số thập phân thành số nhị phân

In [3]:
def dec2bin(decimal):
    """
    Convert decimal to a binary representation.
    
    Parameter
    ---------------
    decimal: int
        Decimal need to be converted to a binary representation
    
    Return
    ---------------
    binary_str: string
        Binary representation of decimal
    """
    binary_str = ''
    if (decimal == 0):
        binary_str = '0'
    else:
        while (decimal > 0):
            binary_str = str(decimal % 2) + binary_str
            decimal /= 2
    
    return binary_str

### 4.2. Hàm rút trích

In [4]:
def extract_lsb(stego_img_file, num_lsbs, extracted_msg_file):
    """
    Extracts the message from a stego image using the LSB method.
    
    Parameters
    ----------
    stego_img_file : string
        The name of the stego img file.
    num_lsbs : int
        The number of LSBs used to embed.
    extracted_msg_file : string
        The name of the extracted message file.
    """
    # TODO
    # 1. Read stego img file
    stego_img = imread(stego_img_file)
        
    # 2. Extract msg bits str from stego img
    # 2.1. Take msg img from stego img and num_lsbs
    msg_img = stego_img - (stego_img >> num_lsbs << num_lsbs)
   
    # 2.2. Convert msg img to msg bits
    msg_img = msg_img.reshape(msg_img.size)
    len_msg_img = len(msg_img)
    msg_bits_str = ''
    for i in range(0, len_msg_img):
        msg_bits = dec2bin(msg_img[i])
        msg_bits_str += '0' * (num_lsbs - len(msg_bits)) + msg_bits
    
    # 2.3. Find index of the last bit 1 of msg bits str,
    # create a new string with content similar to the old string
    # but eliminate the last bits '1000 ...' based on the index of the last bit 1
    index = len(msg_bits_str) - 1     
    while index >= 0:
        if msg_bits_str[index] == '1':
            break
        index -= 1
    msg_bits_str = msg_bits_str[0:index]
    
    # 3. Convert msg bits to msg text
    msg_bits = bitarray.bitarray(msg_bits_str)
    msg_text = msg_bits.tobytes()
    
    # 4. Write msg text to file
    f = open(extracted_msg_file, 'w')
    f.write(msg_text)
    f.close()
    
    return

## 5. Chạy thí nghiệm

Đầu tiên, bạn sẽ gọi hàm nhúng và hàm rút trích ở trên với file cover `cover.bmp` và file message `msg.txt` mà mình đính kèm; bạn sử dụng `num_lsbs = 1`, `opap = True` và để file stego có đuôi là `*.bmp` (để tránh bị mất mát thông tin khi ghi ảnh xuống file). Để minh chứng hàm nhúng và hàm rút trích đã được cài đặt đúng, bạn cũng cần mở file message ban đầu và file message được rút trích, và in nội dung của hai file này ra.

In [5]:
# TODO
input_1 = 'cover.bmp'
input_2 = 'msg.txt'
output_1 = 'stego.bmp'
num_lsbs = 1
opap = True

t = embed_lsb(input_1, input_2, num_lsbs, output_1, opap)
if (t == True):
    output_2 = 'extracted.txt'
    extract_lsb(output_1, num_lsbs, output_2)
    
    f = open(input_2, 'r')
    msg_text = f.read()
    f.close()
    
    f = open(output_2, 'r')
    extracted_text = f.read()
    f.close()
    
    print msg_text
    print '---------------------------------'
    print extracted_text

MAE before OPAP: 0.501163482666 , MAE after OPAP: 0.501163482666 , case 1: 0 % , case 4: 0 %
OVER YOU
[Watch it here: 
https://www.youtube.com/watch?v=y24DLOdFpt0 (by Cassadee Pope)
https://www.youtube.com/watch?v=cGABJorRJ50 (by Blake Shelton)]

Weather man said it iss gonna snow
By now I should be used to the cold
Mid-February shouldnt be so scary
It was only December
I still remember the presents, the tree, you and me

But you went away
How dare you?
I miss you
They say I will be OK
But Im not going to ever get over you

Living alone here in this place
I think of you, and Im not afraid
Your favorite records make me feel better
Cause you sing along
With every song
I know you didnt mean to give them to me

But you went away
How dare you?
I miss you
They say Ill be OK
But Im not going to ever get over you
---------------------------------
OVER YOU
[Watch it here: 
https://www.youtube.com/watch?v=y24DLOdFpt0 (by Cassadee Pope)
https://www.youtube.com/watch?v=cGABJorRJ50 (by Blake Shelto

Kế đến, bạn sẽ viết một đoạn code để thử nghiệm hàm nhúng với `num_lsbs = 1, 2, 3, 4, 5, 6, 7, 8`, và `opap = True` (hàm nhúng của bạn cần in ra các kết quả như ở phần mô tả hàm bên trên). Cho nhận xét về kết quả.

In [6]:
# TODO
input_1 = 'cover.bmp'
input_2 = 'msg.txt'
output_1 = 'stego.bmp'
opap = True

for num_lsbs in range(1, 9):
    t = embed_lsb(input_1, input_2, num_lsbs, output_1, opap)

MAE before OPAP: 0.501163482666 , MAE after OPAP: 0.501163482666 , case 1: 0 % , case 4: 0 %
MAE before OPAP: 1.49826812744 , MAE after OPAP: 0.99951171875 , case 1: 0 % , case 4: 24 %
MAE before OPAP: 3.48915100098 , MAE after OPAP: 2.00187683105 , case 1: 0 % , case 4: 37 %
MAE before OPAP: 7.47224807739 , MAE after OPAP: 3.91215133667 , case 1: 0 % , case 4: 43 %
MAE before OPAP: 15.5174255371 , MAE after OPAP: 8.16014099121 , case 1: 0 % , case 4: 46 %
MAE before OPAP: 31.7149276733 , MAE after OPAP: 16.1750488281 , case 1: 0 % , case 4: 48 %
MAE before OPAP: 58.6238059998 , MAE after OPAP: 34.8123130798 , case 1: 0 % , case 4: 32 %
MAE before OPAP: 123.810417175 , MAE after OPAP: 123.810417175 , case 1: 0 % , case 4: 0 %


TODO: nhận xét
1. Về độ lỗi MAE: Trong hầu hết trường hợp, độ lỗi sau OPAP đều có xu hướng giảm hơn nhiều so với độ lỗi trước OPAP.
2. Tỉ lệ phần trăm case 1 và 4:
    
    Thống kê cho thấy tỉ lệ phần trăm trong trường hợp 1 rất thấp, luôn chạm ngưỡng 0%, còn tỉ lệ phần trăm trong trường hợp 4 luôn duy trì ở mức hai con số (trừ trường hợp num_lsbs = 1 và num_lsbs = 8).
    
    Dễ thấy tỉ lệ case 4 tăng dần từ num_lsbs = 1 đến 6, chạm đỉnh 48% và giảm dần trong 2 trường còn lại.



##### ---

Enjoy coding :-)